# Procesamiento del Lenguaje Natural - TUIA
---
## Trabajo Práctico 1 - Clasificador de Recomendaciones Recreativas utilizando NLP

Integrantes:
- Alsop Agustín (A-4651/7)
- Hachen Rocío (H-1184/3)

### Instalaciones necesarias

In [ ]:
!python -m spacy download es_core_news_lg
!pip install sentence_transformers
#!pip install thefuzz python-Levenshtein
#!pip install transformers sentence_transformers
#!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 821.4 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Se recomienda reiniciar el entorno luego de las instalaciones.

## Librerias

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
#import nltk
#from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
#from thefuzz import process
#import requests
#from bs4 import BeautifulSoup
import spacy
from google.colab import output
import time
from IPython.display import clear_output
from datetime import datetime
import pytz

## Funciones


### Identificación de frases problemáticas

In [ ]:
nlp = spacy.load("es_core_news_lg")

frases_peligrosas = [
    "suicidio", "suicidarse", "no quiero vivir", "quiero morir", "me voy a matar",
    "pegarse un tiro", "matarme", "acabar con todo", "lastimarme", "me hago daño",
    "me quiero cortar", "terminarlo todo", "me odio", "quiero desaparecer",
    "no puedo seguir así", "me quiero morir", "ojalá desapareciera",
    "me voy de este mundo", "no le importo a nadie", "no tengo a nadie",
    "nadie se preocupa por mí", "ya no aguanto más", "hasta aquí llegué",
    "mi vida no vale nada", "ojalá no despertara", "no tengo miedo a morir",
    "un descanso eterno", "tirar por un puente", "me quiero tirar por un puente"
]

def redFlagIdentifier(frase : str) -> bool:
    """
    Verifica si una frase ingresada contiene indicios de pensamientos o expresiones relacionadas con autolesiones
    """

    doc_frase = nlp(frase.lower())

    # Verificamos la similitud con las frases peligrosas
    for frase_peligrosa in frases_peligrosas:
        doc_peligroso = nlp(frase_peligrosa)
        if doc_frase.similarity(doc_peligroso) > 0.7:  # Umbral de similitud
            return True

    return False

#### Test de la función

In [ ]:
frases_polemicas = [
    "me voy a suicidar",
    "no quiero vivir más",
    "hoy es un mal día",
    "no aguanto esto",
    "me quiero tirar por un puente",
    "la vida no tiene sentido",
    "todos estarían mejor sin mí",
    "me siento vacío",
    "quiero acabar con todo",
    "ojalá desapareciera",
    "me voy de este mundo",

    "espero que esto termine pronto",
    "me siento atrapado",
    "la vida es demasiado difícil",
    "no tengo a nadie que me quiera",
    "voy a matar a alguien"
]

# Iterar sobre las frases polémicas y verificar si se consideran peligrosas
for frase in frases_polemicas:
    resultado = redFlagIdentifier(frase)
    print(f"Frase: '{frase}' - ¿Peligrosa?: {resultado}")

Frase: 'me voy a suicidar' - ¿Peligrosa?: True
Frase: 'no quiero vivir más' - ¿Peligrosa?: True
Frase: 'hoy es un mal día' - ¿Peligrosa?: False
Frase: 'no aguanto esto' - ¿Peligrosa?: True
Frase: 'me quiero tirar por un puente' - ¿Peligrosa?: True
Frase: 'la vida no tiene sentido' - ¿Peligrosa?: False
Frase: 'todos estarían mejor sin mí' - ¿Peligrosa?: True
Frase: 'me siento vacío' - ¿Peligrosa?: True
Frase: 'quiero acabar con todo' - ¿Peligrosa?: True
Frase: 'ojalá desapareciera' - ¿Peligrosa?: True
Frase: 'me voy de este mundo' - ¿Peligrosa?: True
Frase: 'espero que esto termine pronto' - ¿Peligrosa?: True
Frase: 'me siento atrapado' - ¿Peligrosa?: True
Frase: 'la vida es demasiado difícil' - ¿Peligrosa?: False
Frase: 'no tengo a nadie que me quiera' - ¿Peligrosa?: True
Frase: 'voy a matar a alguien' - ¿Peligrosa?: True


### Detección de emociones

In [ ]:
def analisisEmocion(input_usuario : str, modeloLR, modeloST) -> str :
  """
  Analiza el estado emocional del usuario con base en el texto ingresado,
  utilizando un modelo de regresión logística (modeloLR) para clasificación de emociones
  y el modelo Sentence Transformer (modeloST) para transformar la frase en una representación vectorial.
  """
  new_phrases = [input_usuario]

  # Preprocesamiento y vectorización de las nuevas frases
  new_phrases_lower = [text.lower() for text in new_phrases]
  new_phrases_vectorized = modeloST.encode(new_phrases_lower)

  # Haciendo predicciones con el modelo entrenado
  new_predictions = modeloLR.predict(new_phrases_vectorized)

  # Definimos el mapeo para las etiquetas
  labels = {-1: 'triste', 0: 'neutro', 1: 'feliz'}

  # Mostrando las predicciones junto con las frases
  for text, label in zip(new_phrases, new_predictions):
      print(f"Texto: '{text}'")
      print(f"Clasificación predicha: {labels[label]}\n")

  return new_predictions[0]

### Funciones secundarias

In [ ]:
def yesNoValidator() -> bool:
  """
  Valida respuestas de sí o no ingresadas por el usuario.
  Acepta varias variantes de cada respuesta (como "Y", "Yes", "Si", etc.), verificando que la entrada sea válida antes de continuar.
  """

  rta = input("Responda si o no (Y/N): ")
  yes_answers = ["Y", "y", "Yes", "Si","S","s",":)"]
  no_answers = ["N", "n", "No",":("]
  valid_answers = yes_answers + no_answers
  while rta not in valid_answers:
    print(f'Por favor conteste con las opciones válidas: {valid_answers}')
    rta = input("Responda si o no (Y/N): ")
  if rta in yes_answers:
    return True
  else:
    return False

In [ ]:
def saludo() -> tuple:
  """
  Genera un saludo contextualizado según la hora local de Buenos Aires, Argentina,
  devolviendo un mensaje de buenos días, buenas tardes o buenas noches, junto con la hora exacta en formato de 24 horas.
  """
  local_timezone = pytz.timezone("America/Argentina/Buenos_Aires")
  hora_actual = datetime.now(local_timezone)

  # Determina el saludo basado en la hora
  if 6 <= hora_actual.hour < 12:
      saludo = "¡Buenos días!"
  elif 12 <= hora_actual.hour < 20:
      saludo = "¡Buenas tardes!"
  else:
      saludo = "¡Buenas noches!"

  hora_minutos = hora_actual.strftime("%H:%M")
  return hora_minutos, saludo

In [ ]:
def loading_animation() -> None:
  """
  Esta función genera una animación de carga en la consola utilizando una secuencia de emojis de gatos y puntos, simulando una espera mientras el sistema procesa alguna tarea.
  """
  loading_text = "Espere un momento"
  animation_frames = ["😺", "😸", "😹", "😻", "😼", "😽", "🙀", "😿", "😾"]
  for _ in range(2):
      punto = "."
      for frame in animation_frames:
          punto += "."
          clear_output(wait=True)
          print(f"{loading_text}{punto} \n{frame}")
          time.sleep(0.3)
  print("¡Carga completa!")
  time.sleep(0.3)

In [ ]:
def activitySearch(prompt : str ,modelo ,db : pd.DataFrame ,quantity : int) -> None:
  """
  Esta función busca actividades en una base de datos según la similitud semántica con el texto de entrada (prompt).
  Utiliza generados por un modelo de transformers para identificar las actividades más relevantes.
  """
  input_embeddings = model2.encode(prompt, convert_to_tensor=True)

  db['coincidencia_sbert'] = db['tensor'].apply(lambda x: util.cos_sim(x, input_embeddings).item())

  db = db.sort_values(by='coincidencia_sbert', ascending=False)
  top_3_activity = db[['Title', 'Description', 'coincidencia_sbert']].head(quantity)

  print(f'Búsqueda: {prompt}\n')
  print('Encontré estos resultados: ')
  for index, row in top_3_activity.iterrows():
    print(f"Título: {row['Title']}")
    print(f"Descripción: {row['Description']}")
    print(f"Similitud: {row['coincidencia_sbert']:.4f}\n")
  #db[['Title','Description', 'coincidencia_sbert']]

### Menu

In [ ]:
def respuestaEmocion(emocion_determinada : int, red_flag : bool = False  ) -> None:
  """
  Maneja la respuesta del sistema según la emoción determinada del usuario y si se detectó alguna frase de riesgo (red_flag).
  Basándose en la emoción del usuario, ofrece recomendaciones de actividades que pueden mejorar su estado de ánimo.
  """
  if emocion_determinada == 0:
    print('Parece que el día se mantiene normal 😊. ¿Quieres una recomendación para el día de hoy?')
    rta = yesNoValidator()
    if rta:
      output.clear()
      print('¡Genial! El día se presta para cualquiera de estas actividades, ¿cuál prefieres?\n Ingresá el número de la opción requerida')
      print('1- Películas\n2- Juegos de mesa\n3- Libros\n4- Salir')
      eleccion = input()
      while eleccion not in ['1','2','3','4']:
        output.clear()
        print('Por favor ingrese una opción válida')
        print('¿Cuál prefieres?\n Ingresá el número de la opción requerida 😊.')
        print('1- Películas\n2- Juegos de mesa\n3- Libros\n4- Salir')
        eleccion = input()
      if eleccion == '1':
        print('¡Genial! Cuéntame qué te gustaría ver: ')
        to_search = input()
        activitySearch(to_search,model2,db_films,2)
      elif eleccion == '2':
        print('¡Genial! Cuéntame qué te gustaría jugar: ')
        to_search = input()
        activitySearch(to_search,model2,db_boardgames,2)
      elif eleccion == '3':
        print('¡Genial! Cuéntame qué te gustaría leer: ')
        to_search = input()
        activitySearch(to_search,model2,db_books,2)
      else:
        output.clear()
        print('¡De acuerdo! Más tarde quizás 😊.')
    else:
      output.clear()
      print('¡De acuerdo! Más tarde quizás 😊.')
  elif emocion_determinada == -1:
    if red_flag:
      print('Lamento escuchar eso 🥺🌧')
      print('Recuerda que siempre es importante hablar con alguien antes de tomar decisiones drásticas.')
      print('Vamos a intentar alegrar el día, ¿qué te parece ver una película?✨🤹‍♀️🎥')
    else:
      print('Lamento escuchar eso 🥺🌧. Vamos a intentar alegrar el día, ¿qué te parece ver una película?✨🤹‍♀️🎥')
    rta = yesNoValidator()
    if rta:
      output.clear()
      print('¡Genial! Cuéntame qué te gustaría ver: ')
      to_search = input()
      activitySearch(to_search,model2,db_films,1)
    else:
      output.clear()
      if red_flag:
        print('¡De acuerdo! Más tarde quizás 😊.')
        print('Recuerda que siempre es importante hablar con alguien antes de tomar decisiones drásticas.')
        print('No estas solo. Wilson esta con vos.')
      else:
        print('¡De acuerdo! Más tarde quizás 😊.')
  else:
    print('¡Me alegra oir eso! Entonces, dime algo que te inspire en este momento, y te daré varias actividades para realizar en base a eso ;). ¿Qué te parece?')
    rta = yesNoValidator()
    if rta:
      output.clear()
      print('¡Genial! Cuéntame sobre que quieres ver, leer o jugar: ')
      to_search = input()
      print('####################################################')
      print('Top 3 peliculas que coinciden con lo que queres ver')
      activitySearch(to_search,model2,db_films,3)
      print('####################################################')
      print('Top 3 juegos de mesa que coinciden con lo que queres ver')
      activitySearch(to_search,model2,db_boardgames,3)
      print('####################################################')
      print('Top 3 libros que coinciden con lo que queres ver')
      activitySearch(to_search,model2,db_books,3)
      print('####################################################')

    else:
      output.clear()
      print('¡De acuerdo! Más tarde quizás 😊.')

In [ ]:
def menu(intro : bool =True):
  """
  Inicializa el menú principal del sistema WILSON
  """
  if intro:

    loading_animation()
    output.clear()

    print('==============================================\n|||||||||||||||||TP 1 - NLP|||||||||||||||||||\n==============================================\n')
    time.sleep(0.3)
    print('Integrantes: ')
    time.sleep(0.3)
    print('Alsop Agustín')
    time.sleep(0.3)
    print('Hachen Rocío')
    time.sleep(2)
    output.clear()
    print("=" * 42)
    print( "🌟✨  Bienvenido al sistema WILSON  ✨🌟")
    print("=" * 42)
    time.sleep(2)
    output.clear()

  info_horaria = saludo()

  print(f"Hora actual: {info_horaria[0]}")
  print(f'{info_horaria[1]} Soy WILSON 🤖 te ayudaré a decidir qué hacer hoy.')
  user_input = input('¿Cómo te sientes hoy?\n')
  output.clear()
  respuestaEmocion(analisisEmocion(user_input,modelo_LR,model),redFlagIdentifier(user_input))


In [ ]:
def start(intro=True) -> None:
  """
  Esta función inicia el sistema WILSON llamando a la función menu()
  """
  menu(intro)

## Bases de Datos

### Base de datos para las emociones
No necesita ninguna transformación

In [ ]:
!gdown 1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s

Downloading...
From (original): https://drive.google.com/uc?id=1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s
From (redirected): https://docs.google.com/spreadsheets/d/1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s/export?format=xlsx
To: /content/NLP data base.xlsx
25.7kB [00:00, 37.0MB/s]


In [ ]:
database = '/content/NLP data base.xlsx'
db_emotion = pd.read_excel(database)

In [ ]:
db_emotion['animo'] = db_emotion['animo'].map({
    'triste': -1,
    'neutro': 0,
    'feliz': 1
})

### Preparación de las bases de datos
Películas, Libros, Juegos de mesas

Esto se realiza una sola vez para generar archivos json donde contendran toda la información necesaria para luego simplemente descargando ese json ejecutar la aplicación

#### Descarga

In [ ]:
"""
# Descargamos los 2 datasets
!gdown 1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c
!gdown 1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
"""

'\n# Descargamos los 2 datasets\n!gdown 1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c\n!gdown 1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq\n'

#### Películas

In [ ]:
"""
peliculas = '/content/IMDB-Movie-Data.csv'
db_films = pd.read_csv(peliculas)
db_films
"""

"\npeliculas = '/content/IMDB-Movie-Data.csv'\ndb_films = pd.read_csv(peliculas)\ndb_films\n"

In [ ]:
"""
# Realizamos el embedding de BERT para cada descripción
db_films['tensor'] = db_films['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))

# Convertir la columna de tensores a listas
db_films['tensor'] = db_films['tensor'].apply(lambda x: [float(val) for val in x])

# Exportar a JSON
db_films.to_json('folder/subfolder/db_films.json', orient='records', lines=True)
"""

"\n# Realizamos el embedding de BERT para cada descripción\ndb_films['tensor'] = db_films['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))\n\n# Convertir la columna de tensores a listas\ndb_films['tensor'] = db_films['tensor'].apply(lambda x: [float(val) for val in x])\n\n# Exportar a JSON\ndb_films.to_json('folder/subfolder/db_films.json', orient='records', lines=True)\n"

#### Juegos de mesa

In [ ]:
"""
juego_mesa = '/content/bgg_database.csv'
db_boardgames = pd.read_csv(juego_mesa)
db_boardgames = db_boardgames.rename(columns={"game_name": "Title", "description": "Description"})
db_boardgames
"""

'\njuego_mesa = \'/content/bgg_database.csv\'\ndb_boardgames = pd.read_csv(juego_mesa)\ndb_boardgames = db_boardgames.rename(columns={"game_name": "Title", "description": "Description"})\ndb_boardgames\n'

In [ ]:
"""
# Realizamos el embedding de BERT para cada descripción
db_boardgames['tensor'] = db_boardgames['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))

# Convertir la columna de tensores a listas
db_boardgames['tensor'] = db_boardgames['tensor'].apply(lambda x: [float(val) for val in x])

# Exportar a JSON
db_boardgames.to_json('folder/subfolder/db_boardgames.json', orient='records', lines=True)
"""

"\n# Realizamos el embedding de BERT para cada descripción\ndb_boardgames['tensor'] = db_boardgames['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))\n\n# Convertir la columna de tensores a listas\ndb_boardgames['tensor'] = db_boardgames['tensor'].apply(lambda x: [float(val) for val in x])\n\n# Exportar a JSON\ndb_boardgames.to_json('folder/subfolder/db_boardgames.json', orient='records', lines=True)\n"

#### Libros

In [ ]:
"""
#URL de la página de Gutenberg con el top 1000 libros
url = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(url)

# Verificamos si la respuesta fue exitosa
if response.status_code == 200:

    # Obtenemos los el contenido de la página, especificamente donde estan todos los titulos
    soup = BeautifulSoup(response.text, 'html.parser')
    books_list = soup.select('body > div.container > div > ol:nth-child(6) > li > a')

    # Lista para almacenar los datos de cada libro
    books_data = []

    # Iteramos sobre cada libro en la lista
    for book in books_list:
        # Extraemos el fragmento donde tiene la url para cargar la página del libro en específico
        book_url = f"https://www.gutenberg.org{book['href']}"

        # Obtener el contenido de la página del libro
        response = requests.get(book_url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscamos la tabla que contiene el título y la descripción
        table = soup.find("table", {"class": "bibrec"})

        # Extraemos el libro, buscando que el header sea Title
        title_row = table.find("th", text="Title")
        if title_row:
            title = title_row.find_next_sibling("td").text.strip()
        else:
            title = "No se encontró el título."

        # Extraemos el resumen, buscando que el header sea Summary
        summary_row = table.find("th", text="Summary")
        if summary_row:
            summary = summary_row.find_next_sibling("td").text.strip()
        else:
            summary = "No se encontró el resumen."

        # Cargamos el título y la descripción a una lista de diccionarios
        books_data.append({
            'Title': title,
            'Description': summary
        })

    # Creamos un DataFrame con los datos recopilados
    db_books = pd.DataFrame(books_data)

else:
    print(f"Error al acceder a la página principal. Código de estado: {response.status_code}")
"""

'\n#URL de la página de Gutenberg con el top 1000 libros\nurl = \'https://www.gutenberg.org/browse/scores/top1000.php#books-last1\'\nresponse = requests.get(url)\n\n# Verificamos si la respuesta fue exitosa\nif response.status_code == 200:\n\n    # Obtenemos los el contenido de la página, especificamente donde estan todos los titulos\n    soup = BeautifulSoup(response.text, \'html.parser\')\n    books_list = soup.select(\'body > div.container > div > ol:nth-child(6) > li > a\')\n\n    # Lista para almacenar los datos de cada libro\n    books_data = []\n\n    # Iteramos sobre cada libro en la lista\n    for book in books_list:\n        # Extraemos el fragmento donde tiene la url para cargar la página del libro en específico\n        book_url = f"https://www.gutenberg.org{book[\'href\']}"\n\n        # Obtener el contenido de la página del libro\n        response = requests.get(book_url)\n        soup = BeautifulSoup(response.content, "html.parser")\n\n        # Buscamos la tabla que cont

In [ ]:
"""
# Realizamos el embedding de BERT para cada descripción
db_books['tensor'] = db_books['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))

# Convertir la columna de tensores a listas
#db_books['tensor'] = db_books['tensor'].apply(lambda x: [float(val) for val in x])

# Exportar a JSON
#db_books.to_json('folder/subfolder/db_books.json', orient='records', lines=True)
"""

"\n# Realizamos el embedding de BERT para cada descripción\ndb_books['tensor'] = db_books['Description'].apply(lambda x: model2.encode(x, convert_to_tensor=True))\n\n# Convertir la columna de tensores a listas\n#db_books['tensor'] = db_books['tensor'].apply(lambda x: [float(val) for val in x])\n\n# Exportar a JSON\n#db_books.to_json('folder/subfolder/db_books.json', orient='records', lines=True)\n"

### Carga de base de datos ya tranformadas
Películas, Libros, Juegos de mesas

#### Descarga

In [ ]:
!gdown 1lzZgGVRZ6LhiZCNT06OhqIUjcrhFCqmo
!gdown 1A9bzLQ1SOBja8_6TiOnGADHtTAjcOaXC
!gdown 1TaqmkromGRIauqH6KV4MdNhGtsw7QaoJ

Downloading...
From: https://drive.google.com/uc?id=1lzZgGVRZ6LhiZCNT06OhqIUjcrhFCqmo
To: /content/db_films.json
100% 7.31M/7.31M [00:00<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A9bzLQ1SOBja8_6TiOnGADHtTAjcOaXC
To: /content/db_boardgames.json
100% 8.96M/8.96M [00:00<00:00, 59.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TaqmkromGRIauqH6KV4MdNhGtsw7QaoJ
To: /content/db_books.json
100% 8.12M/8.12M [00:00<00:00, 36.6MB/s]


#### Películas

In [ ]:
db_films = pd.read_json('/content/db_films.json', orient='records', lines=True)
db_films['tensor'] = db_films['tensor'].apply(lambda x: torch.tensor(x))

#### Juego de Mesa

In [ ]:
db_boardgames = pd.read_json('/content/db_boardgames.json', orient='records', lines=True)
db_boardgames['tensor'] = db_boardgames['tensor'].apply(lambda x: torch.tensor(x))

#### Libros

In [ ]:
db_books = pd.read_json('/content/db_books.json', orient='records', lines=True)
db_books['tensor'] = db_books['tensor'].apply(lambda x: torch.tensor(x))

## Creación del modelo para el estado de ánimo

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
X = db_emotion["frase"].str.lower()
y = db_emotion["animo"].to_numpy() # Convertimos a numpy array

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train.tolist())
X_test_vectorized = model.encode(X_test.tolist())

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.7472527472527473
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

          -1       0.84      0.73      0.78        37
           0       0.69      0.71      0.70        28
           1       0.70      0.81      0.75        26

    accuracy                           0.75        91
   macro avg       0.74      0.75      0.74        91
weighted avg       0.76      0.75      0.75        91



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## Metodología de selección de un juego, pelicula o libro

### SBERT

In [ ]:
model2 = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Código Principal

In [ ]:
start()

¡Genial! Cuéntame sobre que quieres ver, leer o jugar: 
una cancion 
####################################################
Top 3 peliculas que coinciden con lo que queres ver
Búsqueda: una cancion 

Encontré estos resultados: 
Título: Happy Feet
Descripción: Into the world of the Emperor Penguins, who find their soul mates through song, a penguin is born who cannot sing. But he can tap dance something fierce!
Similitud: 0.2058

Título: Mamma Mia!
Descripción: The story of a bride-to-be trying to find her real father told using hit songs by the popular '70s group ABBA.
Similitud: 0.1755

Título: Begin Again
Descripción: A chance encounter between a disgraced music-business executive and a young singer-songwriter new to Manhattan turns into a promising collaboration between the two talents.
Similitud: 0.1688

####################################################
Top 3 juegos de mesa que coinciden con lo que queres ver
Búsqueda: una cancion 

Encontré estos resultados: 
Título: HITSTER
Desc